In [1]:
import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class NnA2CController(nn.Module):
    def __init__(self, frames_shape):
        super(NnA2CController, self).__init__()
        
        #lenet inspired net upscaled due to carla frames being bigger than minst digits ;)
        self.conv_net = nn.Sequential(
            nn.Conv2d(in_channels=frames_shape, out_channels=128, kernel_size=8, stride=4),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=4),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size = 8, stride = 4),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=256, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )
        
        #TODO: policy net, value net, forward method.